# Training a neural network on MNIST with Keras

This simple example demonstrates how to plug TensorFlow Datasets (TFDS) into a Keras model.


Copyright 2020 The TensorFlow Datasets Authors, Licensed under the Apache License, Version 2.0

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/datasets/keras_example"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/datasets/blob/master/docs/keras_example.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/datasets/blob/master/docs/keras_example.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/datasets/docs/keras_example.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

In [1]:
%pip install tensorflow
%pip install tensorflow-datasets
import tensorflow as tf
import tensorflow_datasets as tfds


     -------------------------------------- 376.9/376.9 MB 1.9 MB/s eta 0:00:00
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached gast-0.5.4-py3-none-any.whl (19 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl (7.7 kB)
  Using cached protobuf-4.25.3-cp310-abi3-win_amd64.whl (413 kB)
  Using cached keras-3.3.3-py3-none-any.whl (1.1 MB)
  Using cached requests-2.32.3-py3-none-any.whl (64 kB)
     ---------------------------------------- 15.8/15.8 MB 2.9 MB/s eta 0:00:00
     -------------------------------------- 127.8/127.8 kB 3.7 MB/s eta 0:00:00
  Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
     ---------------------------------------- 4.1/4.1 MB 3.6 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 4.3 MB/s eta 0:00:00
  Using cached tensorboard-2.16.2-py3-none-any.whl (5.5 MB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl (26 kB)
     ---------------------------------------- 3.0/3.0 MB 3.9 MB/s eta 0:00:00
  U

## Step 1: Create your input pipeline

Start by building an efficient input pipeline using advices from:
* The [Performance tips](https://www.tensorflow.org/datasets/performances) guide
* The [Better performance with the `tf.data` API](https://www.tensorflow.org/guide/data_performance#optimize_performance) guide


### Load a dataset

Load the MNIST dataset with the following arguments:

* `shuffle_files=True`: The MNIST data is only stored in a single file, but for larger datasets with multiple files on disk, it's good practice to shuffle them when training.
* `as_supervised=True`: Returns a tuple `(img, label)` instead of a dictionary `{'image': img, 'label': label}`.

In [2]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

c:\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...: 100%|██████████| 4/4 [00:04<00:00,  1.00 url/s]

Dl Completed...: 100%|██████████| 4/4 [00:04<00:00,  1.12s/ url]


Dataset mnist downloaded and prepared to C:\Users\djojo\tensorflow_datasets\mnist\3.0.1. Subsequent calls will reuse this data.


### Build a training pipeline

Apply the following transformations:

* `tf.data.Dataset.map`: TFDS provide images of type `tf.uint8`, while the model expects `tf.float32`. Therefore, you need to normalize images.
* `tf.data.Dataset.cache` As you fit the dataset in memory, cache it before shuffling for a better performance.<br/>
__Note:__ Random transformations should be applied after caching.
* `tf.data.Dataset.shuffle`: For true randomness, set the shuffle buffer to the full dataset size.<br/>
__Note:__ For large datasets that can't fit in memory, use `buffer_size=1000` if your system allows it.
* `tf.data.Dataset.batch`: Batch elements of the dataset after shuffling to get unique batches at each epoch.
* `tf.data.Dataset.prefetch`: It is good practice to end the pipeline by prefetching [for performance](https://www.tensorflow.org/guide/data_performance#prefetching).

In [3]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

### Build an evaluation pipeline

Your testing pipeline is similar to the training pipeline with small differences:

 * You don't need to call `tf.data.Dataset.shuffle`.
 * Caching is done after batching because batches can be the same between epochs.

In [4]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

## Step 2: Create and train the model

Plug the TFDS input pipeline into a simple Keras model, compile the model, and train it.

In [5]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

# Evaluate the model
scores = model.evaluate(ds_test, verbose=0)
print("Keras Accuracy: ", scores[1]*100)

Epoch 1/6


c:\Python\Python310\lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


469/469 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 0.6309 - sparse_categorical_accuracy: 0.8232 - val_loss: 0.1924 - val_sparse_categorical_accuracy: 0.9442
Epoch 2/6
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.1792 - sparse_categorical_accuracy: 0.9487 - val_loss: 0.1341 - val_sparse_categorical_accuracy: 0.9594
Epoch 3/6
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.1200 - sparse_categorical_accuracy: 0.9657 - val_loss: 0.1082 - val_sparse_categorical_accuracy: 0.9673
Epoch 4/6
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0933 - sparse_categorical_accuracy: 0.9737 - val_loss: 0.0938 - val_sparse_categorical_accuracy: 0.9715
Epoch 5/6
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0761 - sparse_categorical_accuracy: 0.9788 - val_loss: 0.0883 - val_sparse_categorical_accuracy: 0.9730
Epoch 6/6
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0568 - sparse_categorical_accuracy: 0.9840 - val_loss: 0.0789 - val_sparse_categorical_accuracy: 0.9746
Keras Accuracy:  97.